In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

rawdata = pd.read_csv("stocks-us-adjClose.csv")

In [2]:
# df1: rename date column appropriately, and convert datatype from string to Timestamp
dict1 = {rawdata.columns[0]: 'date'}

df1 = rawdata.rename(columns=dict1, inplace=False)
df1['date'] = pd.to_datetime(df1['date'])
df_revisions = {'df1': 'date column appropriately named and converted to Timestamp'}
type(df1['date'][0])

pandas._libs.tslib.Timestamp

In [3]:
# df2: create new time columns: year, week, year-week
df2 = df1
df2['year'] = df2['date'].apply(lambda date: date.strftime('%Y'))
df2['week#'] = df2['date'].apply(lambda date: date.strftime('%W'))
df2['year_week#'] = df2['date'].apply(lambda date: date.strftime('%Y wk%W'))
# setting new date columns to appear first:
new_columns_list = ['year', 'year_week#', 'week#']
df2 = df2.set_index(new_columns_list).reset_index()

In [4]:
# groupby: weekly
df_weekly = df2.groupby('year_week#').mean()

In [5]:
plottitle = 'Plot of Data Availability Per Year'
showplot=False

df_annual = df2.groupby('year').mean()
df_years_avail = df_annual.transpose().count()

if showplot:
    fig = plt.figure()
    axes = fig.add_axes()
    plt.plot(df_years_avail)
    plt.title(plottitle)
    plt.ylabel('# of stocks')
    plt.show()

# data prep for a neural network 

### this is very slow, 3 minutes to compile double-loop.
AND, rounding discrepancies exist

In [30]:
t0 = time.time()

tickers = df_weekly.columns

df_weekly_t = df_weekly.transpose()
df_weekly_t_delta = df_weekly_t
    
for ticker in tickers:
    price_series = df_weekly_t_delta.loc[ticker]
    for week in range(len(price_series)-1):
        # Weekly gain/loss:
        #price_series[week+1] = price_series[week+1] - price_series[week]
        price_series[week+1] = format(price_series[week+1] - price_series[week], '.12g')
    price_series.iloc[0] = 0.0
    df_weekly_t_delta.loc[ticker] = price_series
        
t1 = time.time()
t_delta = t1-t0
t_delta = format(t_delta, '.4f')
print("Completed in {} seconds".format(t_delta))
print(df_weekly_t_delta)

Completed in 181.0321 seconds
year_week#  1970 wk00     1970 wk01  1970 wk02  1970 wk03  1970 wk04  \
ED                0.0  7.415200e-03   0.282615  -0.562378   1.140726   
DD                0.0 -6.600000e-06   0.000479  -0.000966   0.001910   
CVX               0.0 -2.000000e-07   0.565780  -1.149677   2.277613   
FL                0.0 -3.521340e-02   1.897905  -3.814024   7.468311   
CAT               0.0 -6.309620e-02   1.565870  -3.147735   6.107070   
IP                0.0  2.954880e-02   1.697311  -1.743408   3.376893   
SJW               0.0           NaN        NaN        NaN        NaN   
F                 0.0           NaN        NaN        NaN        NaN   
LLY               0.0           NaN        NaN        NaN        NaN   
AVP               0.0           NaN        NaN        NaN        NaN   
NFG               0.0           NaN        NaN        NaN        NaN   
BF-A              0.0           NaN        NaN        NaN        NaN   
MATX              0.0           Na

In [ ]:
#CHECKPOINT

In [27]:
def spotcheck(stock, week):
    dfh = df_weekly[stock].head()
    # delta from WEEK 2 to WEEK 1:
    print(format((dfh[week] - dfh[week-1]), '.12g'))

In [31]:
for i in range(4):
    spotcheck('ED', i+1)
'''for i in range(4):
    spotcheck('DD', i+1)
for i in range(4):
    spotcheck('CVX', i+1)
'''
# RESULTS: rounding is off

0.0074152
0.2751994
-0.8449922
1.703104
-6.6e-06
0.0004854
-0.0014444
0.002876
-2.00000000006e-07
0.5657806
-1.715457
3.42729
